In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Step 1: Download historical S&P 500 data
ticker = 'BTC-USD'
data = yf.download(ticker, start='2015-01-01', end='2025-05-01')
returns = 100 * data['Close'].pct_change().dropna()

# Step 2: Fit a GARCH(1,1) model
model = arch_model(returns, vol='Garch', p=1, q=1)
garch_result = model.fit(disp='off')
garch_vol = garch_result.conditional_volatility

# Step 3: Prepare data for Neural Network
# We'll predict next day's volatility using past 10 days of GARCH volatility
look_back = 10
X, y = [], []
for i in range(look_back, len(garch_vol)):
    X.append(garch_vol[i-look_back:i])
    y.append(garch_vol[i])
X, y = np.array(X), np.array(y)

# Split into training and test sets (80% train, 20% test)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Normalize using only training data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

# Reshape for LSTM
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Train the model
model_nn = Sequential()
model_nn.add(LSTM(50, input_shape=(look_back, 1)))
model_nn.add(Dense(1))
model_nn.compile(optimizer='adam', loss='mse')
model_nn.fit(X_train_scaled, y_train_scaled, epochs=20, batch_size=32, verbose=1)

# Predict on test set
predicted_test_scaled = model_nn.predict(X_test_scaled)
predicted_test = scaler_y.inverse_transform(predicted_test_scaled)

# Align predictions with date index
test_index = garch_vol.index[look_back + split:]
predicted_series = pd.Series(predicted_test.flatten(), index=test_index)



[*********************100%***********************]  1 of 1 completed


Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0064
Epoch 2/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0032 
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0025
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0024
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0021
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0026
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0021 
Epoch 8/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0027 
Epoch 9/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0018
Epoch 10/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0018
Epoch 11/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0018 
Epoch 12/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0012  
Epoch 13/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0021
Epoch 14/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0019
Epoch 15/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 
Epoch 16/20


In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=garch_vol.index,
    y=garch_vol,
    mode='lines',
    name='GARCH Volatility',
    line=dict(color='royalblue')
))

fig.add_trace(go.Scatter(
    x=predicted_series.index,
    y=predicted_series,
    mode='lines',
    name='NN Predicted Volatility',
    line=dict(color='orange')
))

fig.update_layout(
    title='GARCH vs Neural Network Volatility Prediction',
    xaxis_title='Date',
    yaxis_title='Volatility',
    template='plotly_white',
    height=600,
    width=1000
)

fig.show()

In [8]:

# 1. Descargar datos del Activo
data = yf.download('BTC-USD', start='2015-01-01')
returns = 100 * data['Close'].pct_change().dropna()

# 2. Calcular volatilidad con GARCH(1,1)
garch_model = arch_model(returns, vol='Garch', p=1, q=1)
garch_result = garch_model.fit(disp='off')
garch_vol = garch_result.conditional_volatility

# 3. Preparar datos para NN
look_back = 10
X, y = [], []
for i in range(look_back, len(garch_vol)):
    X.append(garch_vol[i - look_back:i])
    y.append(garch_vol[i])
X, y = np.array(X), np.array(y)

# Escalado
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# 4. Definir y entrenar la red LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_scaled, y_scaled, epochs=20, batch_size=32, verbose=0)

# 5. Predicción a 10 días hacia el futuro
last_sequence = garch_vol[-look_back:].values
forecast = []

for _ in range(10):
    input_seq = scaler_X.transform(last_sequence.reshape(1, -1)).reshape((1, look_back, 1))
    pred_scaled = model.predict(input_seq, verbose=0)
    pred = scaler_y.inverse_transform(pred_scaled)[0, 0]
    forecast.append(pred)

    # Desplazamos la ventana con el nuevo valor predicho
    last_sequence = np.append(last_sequence[1:], pred)

# Fechas futuras
last_date = garch_vol.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=10, freq='B')
forecast_series = pd.Series(forecast, index=forecast_dates)

# 6. Plot usando Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=garch_vol.index,
    y=garch_vol,
    mode='lines',
    name='GARCH Volatility',
    line=dict(color='royalblue')
))
fig.add_trace(go.Scatter(
    x=forecast_series.index,
    y=forecast_series,
    mode='lines+markers',
    name='Forecast Volatility (NN)',
    line=dict(color='orange', dash='dash')
))

fig.update_layout(
    title='Volatility Forecast (10 days ahead)',
    xaxis_title='Date',
    yaxis_title='Volatility',
    template='plotly_white',
    height=600,
    width=1000
)

fig.show()

[*********************100%***********************]  1 of 1 completed
